In [1]:
import pandas as pd
import numpy as np
import snowflake.connector
from collections import defaultdict
import json
import re
import requests

In [2]:
#login to snowflake db
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')

cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [3]:
#Get list of crawled product variants
sql = '''
select 
distinct (PVID) as PVID,
brand,
_ID,
unique_id,
scraped_attributes :sku sku,
scraped_attributes :item_group_id item_group_id,
scraped_attributes :link link,
scraped_attributes :title title,
scraped_attributes :display_color display_color,
scraped_attributes :size size,
scraped_attributes :options options,
trim (g.value: productIds:: string ,'[]""') AS MATCHED_CI_ID,
--trim (FINAL_PRODUCT_MATCHES: productIds:: string ,'[]""') AS MATCHED_CI_ID,
LAST_CRAWLED_DATE,
status,
FINAL_PRODUCT_MATCHES
from
  PUBLIC.PRODUCTVARIANTS,
  lateral flatten(input => FINAL_PRODUCT_MATCHES) g
where
   brand like '%prismboutique%'
--and status != 'cancelled'
'''
cs.execute(sql)
pvdf = cs.fetch_pandas_all()

In [4]:
#Get list of Customer Inputs
sql = '''
with state as (
  select
    _id,
    CATALOG_ID,
    state
  from
    scale_prod.view.variant_stats_latest_matching
  where
    source_input = 'sent_by_customer'
    and state in (
      'unmatched_no_data_found',
      'unmatched_matching_issue',
      'unmatched_other_reasons'
    )
    and catalog_id in ('399242067345531')
)    
select
  ci.raw_site_url,
  state.catalog_id,
  ci.customer,
  state.state,
  ci._id,
  product_id,
  attribute_data:gtin gtin,
  attribute_data:mfr_part_no mfr_part_no,
  attribute_data:retailer_item_id retailer_item_id, 
  raw_url link,
  attribute_data:name name,
  attribute_data:color :: string color,
  attribute_data:size :: string size,
  attribute_data:gender gender,
  attribute_data:subvertical_attributes sab,
  attribute_data:custom_data :: string cd,
  ci.updated_at

  
from
  state
  join PUBLIC.CUSTOMERINPUTS ci on state.catalog_id = ci.catalog_id
  --join PUBLIC.SITECRAWLS sc on ci.raw_site_url = sc.SITE_URL
  and ci._id = state._id
  and ci.customer = 'flamingo'
--and ci.tags like '%priority-highimp-2022%'
order by catalog_id,name,color,size
'''
cs.execute(sql)

cidf = cs.fetch_pandas_all()


In [ ]:
#cidf.to_csv('burberry.csv')

In [7]:
print(len(cidf),len(pvdf))

5549 43477


In [9]:
pvdf

,PVID,BRAND,_ID,UNIQUE_ID,SKU,ITEM_GROUP_ID,LINK,TITLE,DISPLAY_COLOR,SIZE,OPTIONS,MATCHED_CI_ID,LAST_CRAWLED_DATE,STATUS,FINAL_PRODUCT_MATCHES
0,www.prismboutique.com!32774515359828,www.prismboutique.com,6171d6b6104a63eb730de5d9,32774515359828,"""32774515359828""",4746155425876,"""https://www.prismboutique.com/products/jade-b...","""Jade Bralette""",None,"""XS""","{\n ""Size"": ""XS""\n}",,2022-08-21 10:59:01.551000+00:00,pending_human_job_merge,"[\n {\n ""catalogId"": ""399242067345531"",\n ..."
1,www.prismboutique.com!30399180800084,www.prismboutique.com,62bf473413f00b0cfccef796,30399180800084,"""30399180800084""",4180358955092,"""https://www.prismboutique.com/products/favori...","""Favorite Galaxy Card""",None,None,{},2490773391013984,2022-08-21 11:28:47.533000+00:00,pending_human_job_merge,"[\n {\n ""catalogId"": ""399242067345531"",\n ..."
2,www.prismboutique.com!39507480838228,www.prismboutique.com,62bf4869aabfe7eebb1b0dad,39507480838228,"""39507480838228""",6659828219988,"""https://www.prismboutique.com/products/lavend...","""Lavender Original Overall""",None,"""26""","{\n ""Size"": ""26""\n}",6568974903120144,2022-08-21 11:30:48.521000+00:00,pending_human_job_merge,"[\n {\n ""catalogId"": ""399242067345531"",\n ..."
3,www.prismboutique.com!39363141959764,www.prismboutique.com,617184bcde87b128061f2d3e,39363141959764,"""39363141959764""",6600631550036,"""https://www.prismboutique.com/products/abstra...","""Abstract Shapes Camilla Top""",None,"""S""","{\n ""Size"": ""S""\n}",,2022-08-21 03:42:00.566000+00:00,pending_human_job_merge,"[\n {\n ""catalogId"": ""399242067345531"",\n ..."
4,www.prismboutique.com!39363141959764,www.prismboutique.com,617184bcde87b128061f2d3e,39363141959764,"""39363141959764""",6600631550036,"""https://www.prismboutique.com/products/abstra...","""Abstract Shapes Camilla Top""",None,"""S""","{\n ""Size"": ""S""\n}",6220580381289141,2022-08-21 03:42:00.566000+00:00,pending_human_job_merge,"[\n {\n ""catalogId"": ""399242067345531"",\n ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43472,www.prismboutique.com!28384090685524,www.prismboutique.com,62bf464a13f00b0cfcc57408,28384090685524,"""28384090685524""",3596047450196,"""https://www.prismboutique.com/products/gold-i...","""Gold Ilona Ring""",None,"""6""","{\n ""Size"": ""6""\n}",,2022-08-21 11:24:47.078000+00:00,ready_to_send,"[\n {\n ""catalogId"": ""399242067345531"",\n ..."
43473,www.prismboutique.com!33022706188372,www.prismboutique.com,6171a259104a63eb7381e632,33022706188372,"""33022706188372""",4840584937556,"""https://www.prismboutique.com/products/bronze...","""Saddle Bike Unitard""",None,"""M""","{\n ""Size"": ""M""\n}",,2022-08-21 04:00:29.970000+00:00,pending_human_job_merge,"[\n {\n ""catalogId"": ""399242067345531"",\n ..."
43474,www.prismboutique.com!32985744900180,www.prismboutique.com,6171f1ef104a63eb7349c5ce,32985744900180,"""32985744900180""",4820620804180,"""https://www.prismboutique.com/products/shamar...","""Shamari Alexia Midi Dress""",None,"""L""","{\n ""Size"": ""L""\n}",,2022-08-21 06:20:03.743000+00:00,pending_human_job_merge,"[\n {\n ""catalogId"": ""399242067345531"",\n ..."
43475,www.prismboutique.com!39268249239636,www.prismboutique.com,617186f8de87b128063db830,39268249239636,"""39268249239636""",6546122571860,"""https://www.prismboutique.com/products/aloe-d...","""Aloe Dakota Shorts""",None,"""XS""","{\n ""Size"": ""XS""\n}",,2022-08-21 03:38:17.010000+00:00,pending_human_job_merge,"[\n {\n ""catalogId"": ""399242067345531"",\n ..."


In [ ]:
#get count of inputs to compare
pvcount = len(pvdf.index)
cicount = len(cidf.index)
print (cicount, 'customer inputs need to be matched with', pvcount,'crawled variants, Max achievable match rate =',pvcount/cicount)

In [ ]:
#clean datasets of unnecessary characters, get variant number from url into a new column, extract duplicate customer inputs
cidf['RETAILER_ITEM_ID'] = cidf['RETAILER_ITEM_ID'].map(lambda x: x.lstrip('"').rstrip('"'), na_action='ignore')
cidf['LINK'] = cidf['LINK'].map(lambda x: x.lstrip('"').rstrip('"'), na_action='ignore')
cidf['NAME'] = cidf['NAME'].map(lambda x: x.lstrip('"').rstrip('"'), na_action='ignore')
cidf['GENDER'] = cidf['GENDER'].map(lambda x: x.lstrip('"').rstrip('"'), na_action='ignore')
cidf['PRODUCT_ID'] = cidf['PRODUCT_ID'].map(lambda x: x.lstrip('"').rstrip('"'), na_action='ignore')
cidf['VARIANTNUM'] = cidf['LINK'].str.slice(-14,)

pvdf['SKU'] = pvdf['SKU'].map(lambda x: x.lstrip('"').rstrip('"'), na_action='ignore')
pvdf['ITEM_GROUP_ID'] = pvdf['ITEM_GROUP_ID'].map(lambda x: x.lstrip('"').rstrip('"'), na_action='ignore')
pvdf['LINK'] = pvdf['LINK'].map(lambda x: x.lstrip('"').rstrip('"'), na_action='ignore')
pvdf['TITLE'] = pvdf['TITLE'].map(lambda x: x.lstrip('"').rstrip('"'), na_action='ignore')
pvdf['DISPLAY_COLOR'] = pvdf['DISPLAY_COLOR'].map(lambda x: x.lstrip('"').rstrip('"'), na_action='ignore')
pvdf['SIZE'] = pvdf['SIZE'].map(lambda x: x.lstrip('"').rstrip('"'), na_action='ignore')
pvdf['VARIANTNUM'] = pvdf['LINK'].str.slice(-14,)

#list(cidf.columns)

In [ ]:
#extract duplicate CIs in a new dataframe
duplicatecilink = cidf[cidf.duplicated(['LINK'])]
duplicatecivariantnum = cidf[cidf.duplicated(['VARIANTNUM'])]
cidf['NAMESIZE'] =  cidf['NAME'] + cidf['SIZE']
duplicateciattributes = cidf[cidf.duplicated(['NAMESIZE'])]
#duplicateciattributes.to_csv('duplicateci.csv')

duplinks = len(duplicatecilink.index)
dupvars = len(duplicatecivariantnum.index)
dupatt = len(duplicateciattributes.index)
#print("Duplicate links =", duplinks,"| Duplicate Variant Numbers =", dupvars,"| Duplicate Attributes =", dupatt)

cidf.drop_duplicates(subset=['VARIANTNUM'], keep="first", inplace=True)
dedupcicount = len(cidf.index)

print("Total CIs =", cicount)
print("Dedup CIs to match =", dedupcicount)
#pvdf
#cidf
#duplicatecilink
#duplicatecivariantnum
#duplicateciattributes
#duplicatecilink.to_csv('duplicatecilink.csv')
#duplicatecivariantnum.to_csv('duplicatecivariantnum.csv')
duplicateciattributes.to_csv('duplicateciattributes.csv')

In [ ]:
#apply matching strategy 'urlvariantnumbermatching', match by variant id present in LINK in CI and PVs

#def matchbyurlvariantnumber ():
mergedf = cidf.merge(pvdf,left_on=['VARIANTNUM'],right_on=['VARIANTNUM'],how='left')
mergedf = mergedf.reindex(sorted(mergedf.columns), axis=1)
mergedf = mergedf [['NAME','TITLE','VARIANTNUM', 'COLOR', 'DISPLAY_COLOR', 'SIZE_x', 'SIZE_y', 'LINK_x', 'LINK_y', 'MFR_PART_NO', 'SKU', 'PRODUCT_ID', 'GTIN', 'ITEM_GROUP_ID', 'MATCHED_CI_ID', 'PVID','BRAND', 'CATALOG_ID', 'CD', 'CUSTOMER', 'FINAL_PRODUCT_MATCHES', 'GENDER', 'LAST_CRAWLED_DATE', 'OPTIONS', 'RAW_SITE_URL', 'RETAILER_ITEM_ID', 'SAB', 'STATE', 'STATUS', 'UNIQUE_ID', 'UPDATED_AT', '_ID_x', '_ID_y']]
unmatchcount = mergedf['TITLE'].isna().sum()
unmatcheddf = mergedf[mergedf['TITLE'].isna()]
unmatcheddfc = len(unmatcheddf.index)
matchrate_urlvariantnum = (1-((unmatchcount)/(cicount)))

print("Duplicate CIs=", dupvars)
print("Total CIs=", cicount)
print("Dedup CIs to match=", dedupcicount)
print("CIs left unmatched=", unmatchcount)
print("Number of unmatched CIs =",unmatchcount,"| Strategy Match =",matchrate_urlvariantnum)

#df[df[2].isna()]

#unmatcheddf
#unmatcheddfc
#list(mergedf.columns)
mergedf.to_csv('matched.csv')

In [ ]:
#apply matching strategy 'namesize', match assuming attributes concat lead to unique combinations

#cidf['NAMESIZE']= cidf['NAME'] + cidf['SIZE']
pvdf['NAMESIZE']= pvdf['TITLE'] + pvdf['SIZE']
#mergedf.iloc[0:0]
mergedf = cidf.merge(pvdf,left_on=['NAMESIZE'],right_on=['NAMESIZE'],how='left')
mergedf = mergedf.reindex(sorted(mergedf.columns), axis=1)
mergedf = mergedf [['NAME','TITLE','NAMESIZE', 'COLOR', 'DISPLAY_COLOR', 'SIZE_x', 'SIZE_y', 'LINK_x', 'LINK_y', 'MFR_PART_NO', 'SKU', 'PRODUCT_ID', 'GTIN', 'ITEM_GROUP_ID', 'MATCHED_CI_ID', 'PVID','BRAND', 'CATALOG_ID', 'CD', 'CUSTOMER', 'FINAL_PRODUCT_MATCHES', 'GENDER', 'LAST_CRAWLED_DATE', 'OPTIONS', 'RAW_SITE_URL', 'RETAILER_ITEM_ID', 'SAB', 'STATE', 'STATUS', 'UNIQUE_ID', 'UPDATED_AT', '_ID_x', '_ID_y']]
unmatchcount = mergedf['TITLE'].isna().sum()
matchrate_urlvariantnum = (1-((unmatchcount)/(cicount)))

#print("Duplicate CIs=", dupvars)
print("Total CIs =", cicount)
print("Dedup CIs to match =", dedupcicount)
print("CIs left unmatched =", unmatchcount)
print("Number of unmatched CIs =",unmatchcount,"| Strategy Match =",matchrate_urlvariantnum)

unmatcheddf = mergedf[mergedf['TITLE'].isna()]
unmatcheddfc = len(unmatcheddf.index)
mergedf.to_csv('matched.csv')

In [ ]:
#check for 'No Data Found' or 404s in the unmatched CIs

unmatcheddf.loc[1,'urlresponse'] = "test"
for i, row in unmatcheddf.iterrows():
    try:
        response = requests.get(unmatcheddf.loc[i,'LINK_x'], allow_redirects='False', timeout=5)
        print(response)
        unmatcheddf.at[i,'urlresponse'] = response
    except:
        print("Timeout")
        unmatcheddf.at[i,'urlresponse'] = "Timeout"
    pass


#unmatcheddf


In [ ]:
ndfcount = len(unmatcheddf[(unmatcheddf['urlresponse']=="<Response [404]>")])
notndf = len(unmatcheddf[(unmatcheddf['urlresponse']=="Response [200]")])
timeoutcount = len(unmatcheddf[(unmatcheddf['urlresponse']=="Timeout")])

print(ndfcount,notndf,timeoutcount)

In [ ]:
unmatcheddf.to_csv('urlresponse.csv')